In [38]:
import sys
sys.version

'3.6.8 (default, Jan 14 2019, 11:02:34) \n[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]'

In [39]:
import os
os.chdir("/project/work/Passenger_Demand")

## 패키지 설치
!pip install seaborn
!pip install haversine
!pip install pyarrow
!pip install multiprocessing_on_dill
!pip install statsmodels

In [40]:
import pandas as pd
import numpy as np
import math
import numbers

from haversine import haversine
import seaborn as sns
import matplotlib.pyplot as plt

In [41]:
import datetime

In [42]:
from multiprocessing_on_dill import Pool, cpu_count
from functools import partial
from tqdm import tqdm
import pandas as pd
import numpy as np


def parallelize_dataframe(df, 
                          func, 
                          group_keys = None, 
                          num_cores = None,
                          **params):
    """
        Pandas DataFrame의 apply함수를 병렬처리 하는 함수

        Args: 
            df: 적용 대상 데이터프레임 (Pandas.DataFrame)
            group_keys: apply를 적용할 때 기준이되는 group by key

        Returns:
            df: (Pandas.DataFrame)
            
        Exception: 
    """

    if num_cores is None:
        num_cores = cpu_count()

    if group_keys is None:
        df_list = np.array_split(df, num_cores)
    elif group_keys is not None:
        gr_df = df.groupby(group_keys)
        df_list = [group for name, group in gr_df]
    
    #func = partial(func, **params)
    
    def map_func(data):
        return data.apply(func, axis = 1, group_data= data.copy() ,**params)
        
        
    with Pool(num_cores) as p:

        pd_result = pd.concat(p.map(map_func, df_list))
#         pd_result = p.starmap(func, **params)
        
    return pd_result

In [43]:
def groupby_process(df,
                    func, 
                    group_keys,
                    num_cores = None,
                    **params) :
    
    if num_cores is None:
        num_cores = cpu_count()
        
    gr_df = df.groupby(group_keys)
    df_list = [group for name, group in gr_df]
    
    def map_func(data):
        return func(group_df = data.copy(), **params)
        
    with Pool(num_cores) as p:

        pd_result = pd.concat(p.map(map_func, df_list))
#         pd_result = p.starmap(func, **params)
        
    return pd_result

In [44]:
pd.set_option('display.max_columns', None)

plt.rcParams['font.family'] = 'Nanum Gothic'
sns.set(font="NanumGothic")

In [45]:
%%time
bus_demand_401 = pd.read_parquet('model_mr/mybicard_401_agg.parquet', engine='pyarrow')

CPU times: user 72.5 ms, sys: 41.6 ms, total: 114 ms
Wall time: 39.2 ms


In [50]:
bus_demand_401.groupby(pd.Grouper(key="transdate", freq='1D')).size()

transdate
2020-04-08    1652
2020-04-09    1664
2020-04-10    1732
2020-04-11    1550
2020-04-12    1273
2020-04-13    1673
2020-04-14    1727
2020-04-15       0
2020-04-16       0
2020-04-17       0
2020-04-18       0
2020-04-19       0
2020-04-20    1702
2020-04-21    1692
2020-04-22       0
2020-04-23    1693
2020-04-24    1757
2020-04-25    1483
2020-04-26    1366
2020-04-27    1735
2020-04-28    1738
2020-04-29       0
2020-04-30       0
2020-05-01    1606
2020-05-02    1509
2020-05-03    1315
2020-05-04    1682
2020-05-05       0
2020-05-06    1738
2020-05-07       0
              ... 
2020-11-01       0
2020-11-02    1757
2020-11-03    1771
2020-11-04    1762
2020-11-05    1781
2020-11-06       0
2020-11-07    1594
2020-11-08    1475
2020-11-09       0
2020-11-10       0
2020-11-11       0
2020-11-12       0
2020-11-13       0
2020-11-14       0
2020-11-15       0
2020-11-16       0
2020-11-17    1751
2020-11-18    1787
2020-11-19    1715
2020-11-20    1840
2020-11-21    1659
20

In [46]:
bus_demand_401.sort_values('transdate').head(10)

,mybi_stop_id,transdate,normalcnt,studentcnt,childcnt,totalcnt
223368,3101482,2020-04-08 00:00:00,1,1,0,2
16084,3100144,2020-04-08 00:00:00,2,0,0,2
89825,3101414,2020-04-08 00:00:00,0,1,0,1
164705,3101446,2020-04-08 00:00:00,1,0,0,1
156351,3101442,2020-04-08 00:00:00,1,0,0,1
95714,3101416,2020-04-08 00:00:00,1,0,0,1
150970,3101440,2020-04-08 00:00:00,1,0,0,1
278482,3101537,2020-04-08 05:00:00,2,0,0,2
293924,3101542,2020-04-08 05:00:00,16,0,0,16
13909,3100101,2020-04-08 05:00:00,1,0,0,1


In [9]:
bus_demand_401_check = bus_demand_401.copy()
bus_demand_401_check['date'] = bus_demand_401['transdate'].dt.date

In [10]:
bus_demand_401_check.groupby(['mybi_stop_id']).nunique().head(10)

,mybi_stop_id,transdate,normalcnt,studentcnt,childcnt,totalcnt,date
mybi_stop_id,,,,,,,
3100020,1,2797,15,6,1,16,190
3100021,1,3311,25,5,3,23,190
3100057,1,1518,19,3,2,19,190
3100058,1,2102,20,4,3,19,190
3100085,1,1150,6,6,2,8,189
3100086,1,3031,15,5,4,14,190
3100101,1,1444,8,4,1,8,190
3100106,1,37,4,2,1,3,35
3100142,1,315,7,21,1,22,140


In [11]:
#처음날짜, 마지막날짜
min_date, max_date = bus_demand_401['transdate'].min(), bus_demand_401['transdate'].max()
min_date, max_date

(Timestamp('2020-04-08 00:00:00'), Timestamp('2020-11-30 23:00:00'))

In [12]:
#전체 일수 계산
( bus_demand_401['transdate'].max() - bus_demand_401['transdate'].min() ).days

236

In [13]:
#전체 날짜 생성
entire_date = [min_date + datetime.timedelta(days= i) for i in range(237)]
entire_date[:1], entire_date[-1:]

([Timestamp('2020-04-08 00:00:00')], [Timestamp('2020-11-30 00:00:00')])

In [14]:
#보유데이터 시각 추출
hours = bus_demand_401['transdate'].dt.hour.unique()
hours.sort(); hours

array([ 0,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23])

In [15]:
#전체 transdate 생성
transdate = [x + datetime.timedelta(hours= int(y)) for x in entire_date for y in hours]
transdate[18:22]

[Timestamp('2020-04-08 22:00:00'),
 Timestamp('2020-04-08 23:00:00'),
 Timestamp('2020-04-09 00:00:00'),
 Timestamp('2020-04-09 05:00:00')]

In [16]:
#보유데이터 정류장번호 추출
stop_ids = bus_demand_401['mybi_stop_id'].unique()
len(stop_ids)

131

In [17]:
data = {'mybi_stop_id' : [stop_id for stop_id in stop_ids for i in range(len(transdate))],
        'transdate' : transdate * len(stop_ids)}
bus_stop_df = pd.DataFrame(data= data)

In [18]:
bus_stop_df.head(3)

,mybi_stop_id,transdate
0,3100020,2020-04-08 00:00:00
1,3100020,2020-04-08 05:00:00
2,3100020,2020-04-08 06:00:00


In [19]:
bus_demand_401.head(3)

,mybi_stop_id,transdate,normalcnt,studentcnt,childcnt,totalcnt
0,3100020,2020-04-08 06:00:00,2,0,0,2
1,3100020,2020-04-08 08:00:00,7,0,0,7
2,3100020,2020-04-08 10:00:00,2,0,0,2


In [20]:
aa = pd.merge(bus_stop_df, bus_demand_401, on=['mybi_stop_id', 'transdate'], how='left')

In [21]:
aa.shape

(620940, 6)

In [22]:
def fill_blank_data(data, group_data):
    data_copy = data.copy()
    # 데이터 있는지 확인
    if math.isnan(data_copy['totalcnt']) : # 데이터 없음
        col_names = ['normalcnt','studentcnt','childcnt','totalcnt']
        subset_df = group_data.sort_values(['transdate']).reset_index(drop=True).copy()

        # 해당 일의 데이터가 있는지 확인
        check_day_df = subset_df[subset_df['transdate'].dt.date == data_copy['transdate'].date()][col_names].copy()
        
        if len(check_day_df) == 0 : # 해당 일의 데이터 없음
            
            # 4주 전까지의 데이터 확인
            date_4W_list = [data_copy['transdate'] - datetime.timedelta(weeks= i) for i in range(1,5)]
            if len(date_4W_list) == 0 : # 4주 전까지의 데이터 없음
                tmp_date = data_copy['transdate'].date()
                
                # n주 전 데이터 찾기
                while tmp_date < subset_df['transdate'].min() :
                    result_data = subset_df[subset_df['transdate'] == tmp_date][col_names].copy()
                    if len(result_data) == 0 :
                        tmp_date -= datetime.timedelta(weeks= 1)
                    else :
                        break
                        
                # n주 전 데이터 찾았는지 확인
                if len(result_data) == 0: # False
                    result_data = pd.DataFrame([[data_copy['mybi_stop_id'], data_copy['transdate'], 0, 0, 0, 0]], columns= subset_df.columns)
                    
            else : # 4주 전까지의 데이터 있음
                result_data = subset_df[subset_df['transdate'].isin(date_4W_list)].groupby('mybi_stop_id').mean() \
                            .apply(lambda x: math.ceil(x), axis=0).T[col_names].copy()
                
        else : # 해당 일의 데이터 있음 => 해당 시간대에는 0으로 처리
            result_data = pd.DataFrame([[data_copy['mybi_stop_id'], data_copy['transdate'], 0, 0, 0, 0]], columns= subset_df.columns)
            
    else : # 데이터 있음
        result_data = pd.DataFrame([data_copy])
    return result_data.reset_index(drop=True).loc[0]

In [23]:
%%time
bb = parallelize_dataframe(df = aa, func = fill_blank_data, group_keys = 'mybi_stop_id', num_cores = 40)

CPU times: user 4.89 s, sys: 5.93 s, total: 10.8 s
Wall time: 25min 56s


In [24]:
bb.shape

(620940, 6)

In [25]:
bb.head(10)

,mybi_stop_id,transdate,normalcnt,studentcnt,childcnt,totalcnt
0,3100020,2020-04-08 00:00:00,0.0,0.0,0.0,0.0
1,3100020,2020-04-08 05:00:00,0.0,0.0,0.0,0.0
2,3100020,2020-04-08 06:00:00,2.0,0.0,0.0,2.0
3,3100020,2020-04-08 07:00:00,0.0,0.0,0.0,0.0
4,3100020,2020-04-08 08:00:00,7.0,0.0,0.0,7.0
5,3100020,2020-04-08 09:00:00,0.0,0.0,0.0,0.0
6,3100020,2020-04-08 10:00:00,2.0,0.0,0.0,2.0
7,3100020,2020-04-08 11:00:00,2.0,0.0,0.0,2.0
8,3100020,2020-04-08 12:00:00,1.0,0.0,0.0,1.0
9,3100020,2020-04-08 13:00:00,2.0,1.0,0.0,3.0


In [26]:
bb.isnull().sum()

mybi_stop_id    0
transdate       0
normalcnt       0
studentcnt      0
childcnt        0
totalcnt        0
dtype: int64

In [130]:
def period_before_cnt(group_df, cols, i):
    # i일 전의 lag데이터를 가져옴
    group_df_copy = group_df.sort_values(['transdate']).reset_index(drop=True).copy()
    
    ## cols = ['normalcnt','studentcnt','childcnt','totalcnt']
    for col in cols:
        col_name_day = col + '_D-' + str(i)
        group_df_copy[col_name_day] = group_df_copy.groupby('mybi_stop_id')[col].shift(i*20)
        group_df_copy[col_name_day] = np.where(pd.notnull(group_df_copy[col_name_day]), group_df_copy[col_name_day], group_df_copy[col])
        
        col_name_week = col + '_W-' + str(i)
        group_df_copy[col_name_week] = group_df_copy.groupby('mybi_stop_id')[col].shift(i*20*7)
        group_df_copy[col_name_week] = np.where(pd.notnull(group_df_copy[col_name_week]), group_df_copy[col_name_week], group_df_copy[col])
            
    return group_df_copy

In [131]:
target_cols = ['normalcnt', 'studentcnt', 'childcnt', 'totalcnt']

In [133]:
%%time
mybicard_401_agg_imputation = pd.read_parquet('model_mr/mybicard_401_agg_imputation.parquet', engine='pyarrow')

CPU times: user 603 ms, sys: 1.13 s, total: 1.74 s
Wall time: 411 ms


In [134]:
%%time
aa = groupby_process(df= mybicard_401_agg_imputation[['mybi_stop_id', 'transdate'] + target_cols], func= period_before_cnt, group_keys= 'mybi_stop_id', num_cores= 10
                       , cols= target_cols, i= 1)

CPU times: user 624 ms, sys: 949 ms, total: 1.57 s
Wall time: 2 s


In [135]:
%%time
bb = groupby_process(df= aa, func= period_before_cnt, group_keys= 'mybi_stop_id', num_cores= 10
                       , cols= target_cols, i= 2)

CPU times: user 819 ms, sys: 1.09 s, total: 1.91 s
Wall time: 2.29 s


In [136]:
%%time
cc = groupby_process(df= bb, func= period_before_cnt, group_keys= 'mybi_stop_id', num_cores= 10
                       , cols= target_cols, i= 3)

CPU times: user 1.01 s, sys: 1.34 s, total: 2.35 s
Wall time: 2.7 s


In [137]:
%%time
dd = groupby_process(df= cc, func= period_before_cnt, group_keys= 'mybi_stop_id', num_cores= 10
                       , cols= target_cols, i= 4)

CPU times: user 1.06 s, sys: 1.38 s, total: 2.44 s
Wall time: 2.88 s


In [138]:
%%time
ee = groupby_process(df= dd, func= period_before_cnt, group_keys= 'mybi_stop_id', num_cores= 10
                       , cols= target_cols, i= 5)

CPU times: user 1.26 s, sys: 1.61 s, total: 2.87 s
Wall time: 3.18 s


In [139]:
%%time
ff = groupby_process(df= ee, func= period_before_cnt, group_keys= 'mybi_stop_id', num_cores= 10
                       , cols= target_cols, i= 6)

CPU times: user 1.29 s, sys: 1.48 s, total: 2.77 s
Wall time: 3.02 s


1) 이전 n개일자들의 동일 시간대 평균
2) n주전까지의 동일 요일의 동일 시간대 평균

3) 이전 n개일자들의 전체 평균
4) n주전까지의 동일 요일의 전체 평균
5) n주전까지의 전체 평균

In [157]:
def moving_average_hour(group_df, cols, i):
    group_df_copy = group_df.sort_values(['transdate']).reset_index(drop=True).copy()
    
    ## cols = ['normalcnt', 'studentcnt', 'childcnt', 'totalcnt']
    for col in cols:
        tmp_df = pd.DataFrame()
        mean_col_name = col+'_MA_hour_mean_D'+str(i) # i일전까지의 동일시간대(당일제외)
        std_col_name = col+'_MA_hour_std_D'+str(i) # i일전까지의 동일시간대(당일제외)

        for j in range(1, i+1):
            ## i일전까지의 동일시간대(당일제외) 컬럼생성
            tmp_df[col+'_'+str(j)] = group_df_copy[col].shift(j*20)

        ## 동일시간대 평균컬럼(당일제외) 생성
        ## 동일시간대 평균컬럼(당일제외) 없을 경우 당일집계 데이터 사용
        group_df_copy[mean_col_name] = tmp_df.mean(axis=1)
        group_df_copy[mean_col_name] = np.where(pd.notnull(group_df_copy[mean_col_name]), group_df_copy[mean_col_name], group_df_copy[col])
        # group_df_copy[mean_col_name] = group_df_copy[mean_col_name].apply(np.ceil)
        
        ## 동일시간대 분산컬럼(당일제외) 생성
        ## 동일시간대 분산컬럼(당일제외) 없을 경우 당일집계 데이터 사용
        group_df_copy[std_col_name] = tmp_df.std(axis=1)
        group_df_copy[std_col_name] = np.where(pd.notnull(group_df_copy[std_col_name]), group_df_copy[std_col_name], group_df_copy[col])
        # group_df_copy[mean_col_name] = group_df_copy[mean_col_name].apply(np.ceil)
    
    ## cols = ['normalcnt', 'studentcnt', 'childcnt', 'totalcnt']
    for col in cols:
        tmp_df = pd.DataFrame()
        mean_col_name = col+'_MA_hour_mean_W'+str(i) # i주전까지의 동일시간대(당일제외)
        std_col_name = col+'_MA_hour_std_W'+str(i) # i일전까지의 동일시간대(당일제외)

        for j in range(1, i+1):
            ## i주전까지의 동일시간대(당일제외) 컬럼생성
            tmp_df[col+'_'+str(j)] = group_df_copy[col].shift(j*20*7)
            
        ## 동일시간대 평균컬럼(당일제외) 생성
        ## 동일시간대 평균컬럼(당일제외) 없을 경우 당일집계 데이터 사용
        group_df_copy[mean_col_name] = tmp_df.mean(axis=1)
        group_df_copy[mean_col_name] = np.where(pd.notnull(group_df_copy[mean_col_name]), group_df_copy[mean_col_name], group_df_copy[col])
        # group_df_copy[mean_col_name] = group_df_copy[mean_col_name].apply(np.ceil)
        
        ## 동일시간대 분산컬럼(당일제외) 생성
        ## 동일시간대 분산컬럼(당일제외) 없을 경우 당일집계 데이터 사용
        group_df_copy[std_col_name] = tmp_df.std(axis=1)
        group_df_copy[std_col_name] = np.where(pd.notnull(group_df_copy[std_col_name]), group_df_copy[std_col_name], group_df_copy[col])
        # group_df_copy[mean_col_name] = group_df_copy[mean_col_name].apply(np.ceil)

    return group_df_copy

In [158]:
%%time
aaa = groupby_process(df= ff, func= moving_average_hour, group_keys= 'mybi_stop_id', num_cores= 10
                       , cols= target_cols, i= 2)

CPU times: user 1.88 s, sys: 2.54 s, total: 4.41 s
Wall time: 4.8 s


In [159]:
%%time
bbb = groupby_process(df= aaa, func= moving_average_hour, group_keys= 'mybi_stop_id', num_cores= 10
                       , cols= target_cols, i= 3)

CPU times: user 2.15 s, sys: 2.38 s, total: 4.53 s
Wall time: 4.96 s


In [160]:
%%time
ccc = groupby_process(df= bbb, func= moving_average_hour, group_keys= 'mybi_stop_id', num_cores= 10
                       , cols= target_cols, i= 4)

CPU times: user 2.65 s, sys: 3.16 s, total: 5.81 s
Wall time: 6.1 s


In [161]:
%%time
ddd = groupby_process(df= ccc, func= moving_average_hour, group_keys= 'mybi_stop_id', num_cores= 10
                       , cols= target_cols, i= 5)

CPU times: user 2.95 s, sys: 2.53 s, total: 5.48 s
Wall time: 5.9 s


In [162]:
%%time
eee = groupby_process(df= ddd, func= moving_average_hour, group_keys= 'mybi_stop_id', num_cores= 10
                       , cols= target_cols, i= 6)

CPU times: user 3.12 s, sys: 2.65 s, total: 5.76 s
Wall time: 6.19 s


In [163]:
def moving_average_all(group_df, cols, i):
    group_df_copy = group_df.sort_values(['transdate']).reset_index(drop=True).copy()
    ## 일자별 집계
    group_sum_df_copy = group_df_copy[['transdate'] + cols].groupby(group_df_copy['transdate'].dt.date).sum().copy()

    ## cols = ['normalcnt', 'studentcnt', 'childcnt', 'totalcnt']
    for col in cols:
        tmp_df = pd.DataFrame()
        mean_col_name_all = col+'_MA_all_mean_A'+str(i) # i주전까지의 일집계(당일제외)
        mean_col_name_day = col+'_MA_all_mean_D'+str(i) # i일전까지의 일집계(당일제외)
        mean_col_name_week = col+'_MA_all_mean_W'+str(i) # i주전까지의 동일요일 일집계(당일제외)
        
        std_col_name_all = col+'_MA_all_std_A'+str(i) # i주전까지의 일집계(당일제외)
        std_col_name_day = col+'_MA_all_std_D'+str(i) # i일전까지의 일집계(당일제외)
        std_col_name_week = col+'_MA_all_std_W'+str(i) # i주전까지의 동일요일 일집계(당일제외)

        for j in range(1, (i*7)+1):
            ## i주 전까지의 일집계(당일제외) 컬럼생성
            tmp_df[col+'_'+str(j)] = group_sum_df_copy[col].shift(j)
        
        ## 일집계 평균컬럼(당일제외) 생성
        group_sum_df_copy[mean_col_name_all] = tmp_df.mean(axis=1)
        group_sum_df_copy[mean_col_name_day] = tmp_df[tmp_df.columns[:i]].mean(axis=1)
        group_sum_df_copy[mean_col_name_week] = tmp_df[[tmp_df.columns[7*n-1] for n in range(1,i)]].mean(axis=1)
        
        ## 일집계 평균데이터가 없을 경우 당일집계 데이터 사용
        group_sum_df_copy[mean_col_name_all] = np.where(pd.notnull(group_sum_df_copy[mean_col_name_all]), group_sum_df_copy[mean_col_name_all], group_sum_df_copy[col])
        group_sum_df_copy[mean_col_name_day] = np.where(pd.notnull(group_sum_df_copy[mean_col_name_day]), group_sum_df_copy[mean_col_name_day], group_sum_df_copy[col])
        group_sum_df_copy[mean_col_name_week] = np.where(pd.notnull(group_sum_df_copy[mean_col_name_week]), group_sum_df_copy[mean_col_name_week], group_sum_df_copy[col])
        
        ## 일집계 평균컬럼(당일제외) 생성
        group_sum_df_copy[std_col_name_all] = tmp_df.std(axis=1)
        group_sum_df_copy[std_col_name_day] = tmp_df[tmp_df.columns[:i]].std(axis=1)
        group_sum_df_copy[std_col_name_week] = tmp_df[[tmp_df.columns[7*n-1] for n in range(1,i)]].std(axis=1)
        
        ## 일집계 평균데이터가 없을 경우 당일집계 데이터 사용
        group_sum_df_copy[std_col_name_all] = np.where(pd.notnull(group_sum_df_copy[std_col_name_all]), group_sum_df_copy[std_col_name_all], group_sum_df_copy[col])
        group_sum_df_copy[std_col_name_day] = np.where(pd.notnull(group_sum_df_copy[std_col_name_day]), group_sum_df_copy[std_col_name_day], group_sum_df_copy[col])
        group_sum_df_copy[std_col_name_week] = np.where(pd.notnull(group_sum_df_copy[std_col_name_week]), group_sum_df_copy[std_col_name_week], group_sum_df_copy[col])
        
        ## 일집계 평균데이터 올림
        # group_sum_df_copy[col_name_all] = group_sum_df_copy[col_name_all].apply(np.ceil)
        # group_sum_df_copy[col_name_day] = group_sum_df_copy[col_name_day].apply(np.ceil)
        # group_sum_df_copy[col_name_week] = group_sum_df_copy[col_name_week].apply(np.ceil)
    
    ## MA데이터 생성
    group_df_copy = pd.merge(group_df_copy, group_sum_df_copy[group_sum_df_copy.columns[4:]]
                            , left_on= group_df_copy['transdate'].dt.date, right_index= True, how= 'left')
    return group_df_copy

In [164]:
%%time
aaaa = groupby_process(df= eee, func= moving_average_all, group_keys= 'mybi_stop_id', num_cores= 10, cols= target_cols, i= 1)

CPU times: user 3.52 s, sys: 3.22 s, total: 6.74 s
Wall time: 7.05 s


In [167]:
%%time
bbbb = groupby_process(df= aaaa, func= moving_average_all, group_keys= 'mybi_stop_id', num_cores= 10, cols= target_cols, i= 2)

CPU times: user 3.71 s, sys: 3.49 s, total: 7.2 s
Wall time: 7.54 s


In [168]:
%%time
cccc = groupby_process(df= bbbb, func= moving_average_all, group_keys= 'mybi_stop_id', num_cores= 10, cols= target_cols, i= 3)

CPU times: user 4.11 s, sys: 3.86 s, total: 7.96 s
Wall time: 8.1 s


In [169]:
%%time
dddd = groupby_process(df= cccc, func= moving_average_all, group_keys= 'mybi_stop_id', num_cores= 10, cols= target_cols, i= 4)

CPU times: user 4.86 s, sys: 6.12 s, total: 11 s
Wall time: 11.3 s


In [170]:
%%time
eeee = groupby_process(df= dddd, func= moving_average_all, group_keys= 'mybi_stop_id', num_cores= 10, cols= target_cols, i= 5)

CPU times: user 5.36 s, sys: 4.26 s, total: 9.61 s
Wall time: 10 s


In [171]:
%%time
ffff = groupby_process(df= eeee, func= moving_average_all, group_keys= 'mybi_stop_id', num_cores= 10, cols= target_cols, i= 6)

CPU times: user 5.89 s, sys: 5.37 s, total: 11.3 s
Wall time: 11.3 s


In [36]:
ffff.to_parquet("model_mr/mybicard_401_agg_moving_average.parquet")

In [23]:
ffff.shape

(620940, 138)

In [27]:
ffff.head(3)

,mybi_stop_id,transdate,normalcnt,studentcnt,childcnt,totalcnt,normalcnt_D-1,studentcnt_D-1,childcnt_D-1,totalcnt_D-1,normalcnt_D-2,studentcnt_D-2,childcnt_D-2,totalcnt_D-2,normalcnt_D-3,studentcnt_D-3,childcnt_D-3,totalcnt_D-3,normalcnt_D-4,studentcnt_D-4,childcnt_D-4,totalcnt_D-4,normalcnt_W-1,studentcnt_W-1,childcnt_W-1,totalcnt_W-1,normalcnt_W-2,studentcnt_W-2,childcnt_W-2,totalcnt_W-2,normalcnt_W-3,studentcnt_W-3,childcnt_W-3,totalcnt_W-3,normalcnt_W-4,studentcnt_W-4,childcnt_W-4,totalcnt_W-4,normalcnt_MA_hour_D2,studentcnt_MA_hour_D2,childcnt_MA_hour_D2,totalcnt_MA_hour_D2,normalcnt_MA_hour_W2,studentcnt_MA_hour_W2,childcnt_MA_hour_W2,totalcnt_MA_hour_W2,normalcnt_MA_hour_D3,studentcnt_MA_hour_D3,childcnt_MA_hour_D3,totalcnt_MA_hour_D3,normalcnt_MA_hour_W3,studentcnt_MA_hour_W3,childcnt_MA_hour_W3,totalcnt_MA_hour_W3,normalcnt_MA_hour_D4,studentcnt_MA_hour_D4,childcnt_MA_hour_D4,totalcnt_MA_hour_D4,normalcnt_MA_hour_W4,studentcnt_MA_hour_W4,childcnt_MA_hour_W4,totalcnt_MA_hour_W4,normalcnt_MA_hour_D5,studentcnt_MA_hour_D5,childcnt_MA_hour_D5,totalcnt_MA_hour_D5,normalcnt_MA_hour_W5,studentcnt_MA_hour_W5,childcnt_MA_hour_W5,totalcnt_MA_hour_W5,normalcnt_MA_hour_D6,studentcnt_MA_hour_D6,childcnt_MA_hour_D6,totalcnt_MA_hour_D6,normalcnt_MA_hour_W6,studentcnt_MA_hour_W6,childcnt_MA_hour_W6,totalcnt_MA_hour_W6,normalcnt_MA_all_A2,normalcnt_MA_all_D2,normalcnt_MA_all_W2,studentcnt_MA_all_A2,studentcnt_MA_all_D2,studentcnt_MA_all_W2,childcnt_MA_all_A2,childcnt_MA_all_D2,childcnt_MA_all_W2,totalcnt_MA_all_A2,totalcnt_MA_all_D2,totalcnt_MA_all_W2,normalcnt_MA_all_A3,normalcnt_MA_all_D3,normalcnt_MA_all_W3,studentcnt_MA_all_A3,studentcnt_MA_all_D3,studentcnt_MA_all_W3,childcnt_MA_all_A3,childcnt_MA_all_D3,childcnt_MA_all_W3,totalcnt_MA_all_A3,totalcnt_MA_all_D3,totalcnt_MA_all_W3,normalcnt_MA_all_A4,normalcnt_MA_all_D4,normalcnt_MA_all_W4,studentcnt_MA_all_A4,studentcnt_MA_all_D4,studentcnt_MA_all_W4,childcnt_MA_all_A4,childcnt_MA_all_D4,childcnt_MA_all_W4,totalcnt_MA_all_A4,totalcnt_MA_all_D4,totalcnt_MA_all_W4,normalcnt_MA_all_A5,normalcnt_MA_all_D5,normalcnt_MA_all_W5,studentcnt_MA_all_A5,studentcnt_MA_all_D5,studentcnt_MA_all_W5,childcnt_MA_all_A5,childcnt_MA_all_D5,childcnt_MA_all_W5,totalcnt_MA_all_A5,totalcnt_MA_all_D5,totalcnt_MA_all_W5,normalcnt_MA_all_A6,normalcnt_MA_all_D6,normalcnt_MA_all_W6,studentcnt_MA_all_A6,studentcnt_MA_all_D6,studentcnt_MA_all_W6,childcnt_MA_all_A6,childcnt_MA_all_D6,childcnt_MA_all_W6,totalcnt_MA_all_A6,totalcnt_MA_all_D6,totalcnt_MA_all_W6
0,3100020,2020-04-08 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,48.0,48.0,2.0,2.0,2.0,0.0,0.0,0.0,50.0,50.0,50.0,48.0,48.0,48.0,2.0,2.0,2.0,0.0,0.0,0.0,50.0,50.0,50.0,48.0,48.0,48.0,2.0,2.0,2.0,0.0,0.0,0.0,50.0,50.0,50.0,48.0,48.0,48.0,2.0,2.0,2.0,0.0,0.0,0.0,50.0,50.0,50.0,48.0,48.0,48.0,2.0,2.0,2.0,0.0,0.0,0.0,50.0,50.0,50.0
1,3100020,2020-04-08 05:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,48.0,48.0,48.0,2.0,2.0,2.0,0.0,0.0,0.0,50.0,50.0,50.0,48.0,48.0,48.0,2.0,2.0,2.0,0.0,0.0,0.0,50.0,50.0,50.0,48.0,48.0,48.0,2.0,2.0,2.0,0.0,0.0,0.0,50.0,50.0,50.0,48.0,48.0,48.0,2.0,2.0,2.0,0.0,0.0,0.0,50.0,50.0,50.0,48.0,48.0,48.0,2.0,2.0,2.0,0.0,0.0,0.0,50.0,50.0,50.0
2,3100020,2020-04-08 06:00:00,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,0.0,0.0,2.0,2.0,

In [35]:
#18시의 같은정류장 2주치의 데이터
ffff[(ffff['mybi_stop_id']==3100020) & (ffff['transdate'].dt.hour==18)] \
    [['mybi_stop_id', 'transdate', 'totalcnt', 'totalcnt_D-1', 'totalcnt_D-2',
      'totalcnt_MA_hour_D2', 'totalcnt_MA_hour_W2',
      'totalcnt_MA_all_A2', 'totalcnt_MA_all_D2', 'totalcnt_MA_all_W2']].head(14)

,mybi_stop_id,transdate,totalcnt,totalcnt_D-1,totalcnt_D-2,totalcnt_MA_hour_D2,totalcnt_MA_hour_W2,totalcnt_MA_all_A2,totalcnt_MA_all_D2,totalcnt_MA_all_W2
14,3100020,2020-04-08 18:00:00,2.0,2.0,2.0,2.0,2.0,50.0,50.0,50.0
34,3100020,2020-04-09 18:00:00,0.0,2.0,0.0,2.0,0.0,50.0,50.0,27.0
54,3100020,2020-04-10 18:00:00,2.0,0.0,2.0,1.0,2.0,39.0,39.0,42.0
74,3100020,2020-04-11 18:00:00,1.0,2.0,0.0,1.0,1.0,40.0,35.0,25.0
94,3100020,2020-04-12 18:00:00,2.0,1.0,2.0,2.0,2.0,36.0,34.0,23.0
114,3100020,2020-04-13 18:00:00,3.0,2.0,1.0,2.0,3.0,34.0,24.0,41.0
134,3100020,2020-04-14 18:00:00,7.0,3.0,2.0,3.0,7.0,35.0,32.0,57.0
154,3100020,2020-04-15 18:00:00,0.0,7.0,3.0,5.0,2.0,38.0,49.0,50.0
174,3100020,2020-04-16 18:00:00,0.0,0.0,7.0,4.0,0.0,34.0,29.0,27.0
194,3100020,2020-04-17 18:00:00,0.0,0.0,0.0,0.0,2.0,30.0,0.0,42.0
